In [1]:
import pandas as pd
import numpy as np
sales = pd.read_csv('sales.csv')
prices = pd.read_csv('prices.csv')

In [2]:
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])
prices['updated_at'] = pd.to_datetime(prices['updated_at'])

In [3]:
#def function that return the closest date that price update
def closest_update_date (id, order_date):
    new_date = prices['updated_at'][prices['product_id'] == id]
    if order_date < new_date.min():
       closest_date = new_date.min()
    else :
       closest_date = new_date[order_date > new_date].max()                            
    return closest_date

In [4]:
#def function that return closest price base on productid and order_date
def return_closest_price(id, order_date):
    day = closest_update_date (id, order_date)
    if order_date < day :
       closest_price = prices['old_price'][(prices['updated_at'] == day) & (prices['product_id'] == id)].values[0]
    else :
       closest_price = prices['new_price'][(prices['updated_at'] == day) & (prices['product_id'] == id)].values[0]                              
    return closest_price

In [5]:
#Find price corresponding each row and append to update_price array.
update_price=[]
for i in np.arange(len(sales)):
    updated_price = return_closest_price(sales.iloc[i]['product_id'], sales.iloc[i]['ordered_at'])
    update_price.append(updated_price)

#create and calculate price and revenue column
sales["price"] = update_price
sales["revenue"] = sales['price'] * sales['quantity_ordered']

#export to csv file
print(sales)



     product_id          ordered_at  quantity_ordered   price  revenue
0       3998909 2018-09-18 17:51:00                 1   16500    16500
1       3998909 2018-09-18 12:52:00                 1   16500    16500
2       3998909 2018-09-18 11:33:00                 1   16500    16500
3       3998909 2018-09-18 18:47:00                 1   16500    16500
4       3998909 2018-09-18 17:36:00                 1   16500    16500
..          ...                 ...               ...     ...      ...
170     4085861 2018-09-13 23:29:00                 1   62500    62500
171          64 2018-09-11 19:32:00                 1  239000   239000
172          64 2018-09-14 20:10:00                 1  239000   239000
173          64 2018-09-12 16:00:00                 1  239000   239000
174          64 2018-09-12 13:23:00                 1  239000   239000

[175 rows x 5 columns]


In [6]:
sales.groupby('product_id').agg({'revenue':sum})

,revenue
product_id,
64,956000
3954203,877500
3998909,280500
4085861,8247500
